Deliverable 3: Pre-processing Data

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
url = "https://raw.githubusercontent.com/laurlen2112/neural_network_charity_analysis-/main/charity_data.csv"
application_df = pd.read_csv(url)
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'**** may need to drop more***
app_df = application_df.drop(["EIN", "NAME"], axis = 1)
app_df.head()

In [ ]:
# Determine the number of unique values in each column (19.4.3)
app_cat = app_df.dtypes[app_df.dtypes == "object"].index.tolist()
app_df[app_cat].nunique()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_bin = app_df["APPLICATION_TYPE"].value_counts()
application_bin

In [ ]:
# Determine which values to replace if counts are less than ...? (19.3.3)
replace_application = list(application_bin[application_bin <500].index)

# Replace in dataframe
for app in replace_application:
    app_df.APPLICATION_TYPE = app_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
app_df.APPLICATION_TYPE.value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning 
app_class_bin = app_df["CLASSIFICATION"].value_counts()
app_class_bin

In [ ]:
# Determine which values to replace if counts are less than ..?
replace_app_class = list(app_class_bin[app_class_bin <1000].index)

# Replace in dataframe
for cls in replace_app_class:
    app_df.CLASSIFICATION = app_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
app_df.CLASSIFICATION.value_counts()

In [ ]:
# Generate our categorical variable lists (19.4.3)
app_cat = app_df.dtypes[app_df.dtypes == "object"].index.tolist()
app_cat

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(app_df[app_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(app_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals (19.3.3)

new_app_df = app_df.merge(encode_df, left_index = True, right_index = True)
new_app_df = new_app_df.drop(columns = app_cat, axis = 1 )
new_app_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays (19.4.3)
y = new_app_df["IS_SUCCESSFUL"].values
x = new_app_df.drop(["IS_SUCCESSFUL"], axis = 1).values

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=78)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

Optimization Attempt 1

Optimization Attempt 2

Optimization Attempt 3